# Advent of Code 2020 Day 1
[link](https://adventofcode.com/2020/day/1)

If going for speed, we can just brute force the answer via nested for-loops. This question's size is quite suitable for brute force. The input is an integer array of length 200. This means that part 1 need about $4 \times 10^4$ checks, while part 2 need about $8 \times 10^6 \approx 2^{23}$ checks. This is very manageable with the speed of our computer.

This is however a commonly seen problem that can be amortized using some sorting and searching algorithm.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#read-input" data-toc-modified-id="read-input-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>read input</a></span></li><li><span><a href="#part-1" data-toc-modified-id="part-1-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>part 1</a></span><ul class="toc-item"><li><span><a href="#solution-1" data-toc-modified-id="solution-1-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>solution 1</a></span></li><li><span><a href="#solution-2" data-toc-modified-id="solution-2-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>solution 2</a></span></li><li><span><a href="#answer" data-toc-modified-id="answer-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>answer</a></span></li></ul></li><li><span><a href="#part-2" data-toc-modified-id="part-2-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>part 2</a></span><ul class="toc-item"><li><span><a href="#solution-1" data-toc-modified-id="solution-1-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>solution 1</a></span></li><li><span><a href="#solution-2" data-toc-modified-id="solution-2-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>solution 2</a></span></li><li><span><a href="#answer" data-toc-modified-id="answer-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>answer</a></span></li></ul></li></ul></div>

## read input

In [1]:
parse_input(filename) = parse.(Int, readlines(filename))

parse_input (generic function with 1 method)

In [2]:
input_sample_1 = parse_input("input_sample_1.txt")

6-element Array{Int64,1}:
 1721
  979
  366
  299
  675
 1456

In [3]:
input_puzzle = parse_input("input_puzzle.txt")

200-element Array{Int64,1}:
 1822
 1917
 1642
 1617
 1941
 1740
 1529
 1896
 1880
  568
 1897
 1521
 1832
    ⋮
 1865
 1535
 1994
 1756
 1662
 1621
 1993
 1825
 1679
 1959
 1691
 1875

## part 1

### solution 1

Brute force with time complexity $O(n^2)$.

To going for speed, this is a sloppier easy to write solution with are a few assumptions, e.g. the entries are distinct, and there is no 1010 (so it ended up add itself to get 2020), etc. Without that, we will need to traverse the indices and not the values of the entries, so that single 1010 is not accepted but double 1010 is.

In [4]:
function two_entries_sum_to_2020(v, ::Val{:solution1})
    for a1 in v, a2 in v
        if a1 + a2 == 2020
            return [a1, a2]
        end
    end
end

two_entries_sum_to_2020 (generic function with 1 method)

### solution 2

A way is to sort the array first with time complexity of $O(n \lg n)$, and then traverse it from both end with time complexity of $O(n)$. It is probably not worth the effort for the size of the input array. This solution handles the edge cases in the previous solution.

In [5]:
function two_entries_sum_to_2020(v, ::Val{:solution2})
    v = sort(v)
    i = 1
    j = length(v)
    while i != j
        total = v[i] + v[j]
        if total == 2020
            return [v[i], v[j]]
        elseif total < 2020
            i += 1
        else
            j -= 1
        end
    end
end

two_entries_sum_to_2020 (generic function with 2 methods)

### answer

In [6]:
function show_answer_report(input, ::Val{:part1}, solution_val)
    @show entries_with_issue = two_entries_sum_to_2020(input, solution_val)
    @show sum(entries_with_issue)
    @info "Answer found." answer=prod(entries_with_issue)
    return
end

show_answer_report (generic function with 1 method)

In [7]:
@time show_answer_report(input_sample_1, Val(:part1), Val(:solution1))

entries_with_issue = two_entries_sum_to_2020(input, solution_val) = [1721, 299]
sum(entries_with_issue) = 2020
  0.286686 seconds (645.32 k allocations: 33.350 MiB, 3.08% gc time)
┌ Info: Answer found.
│   answer = 514579
└ @ Main In[6]:4


In [8]:
@time show_answer_report(input_puzzle, Val(:part1), Val(:solution1))

entries_with_issue = two_entries_sum_to_2020(input, solution_val) = [1245, 775]
sum(entries_with_issue) = 2020
  0.000227 seconds (148 allocations: 4.859 KiB)
┌ Info: Answer found.
│   answer = 964875
└ @ Main In[6]:4


## part 2

### solution 1

Brute force. Similar assumption as previous.

In [9]:
function three_entries_sum_to_2020(v, ::Val{:solution1})
    for a1 in v, a2 in v, a3 in v
        if a1 + a2 + a3 == 2020
            return [a1, a2, a3]
        end
    end
end

three_entries_sum_to_2020 (generic function with 1 method)

### solution 2

A way is to create a lookup of all sum of two entries with time complexity $O(n^2)$, and then for each entry lookup whether there exists a sum that can complete it to 2020 (with time complexity $(n^2)$). Similarly it is probably not worth the effort for the size of the input array.

In this solution we need extra assumptions that no valid 3 sums consist of repeated values, and there isn't more than one pair of entries with the same sum. In that event we will need a multi-dict instead of a dict to store the lookup, and work with indices instead of values for the entries.

In [10]:
function lookup_two_sums(v)
    result = Dict{Int, Vector{Int}}()
    for i in 1:length(v), j in 1:(i - 1)
        result[v[i] + v[j]] = [v[i], v[j]]
    end
    return result
end

lookup_two_sums (generic function with 1 method)

In [11]:
function three_entries_sum_to_2020(v, ::Val{:solution2})
    lookup = lookup_two_sums(v)
    for a in v
        diff = 2020 - a
        if haskey(lookup, diff)
            remaining_entries = lookup[diff]
            (a ∉ remaining_entries) && return [a, lookup[diff]...]
        end
    end
end

three_entries_sum_to_2020 (generic function with 2 methods)

### answer

In [12]:
function show_answer_report(input, ::Val{:part2}, solution_val)
    @show entries_with_issue = three_entries_sum_to_2020(input, solution_val)
    @show sum(entries_with_issue)
    @info "Answer found." answer=prod(entries_with_issue)
    return
end

show_answer_report (generic function with 2 methods)

In [13]:
@time show_answer_report(input_sample_1, Val(:part2), Val(:solution1))

entries_with_issue = three_entries_sum_to_2020(input, solution_val) = [979, 366, 675]
sum(entries_with_issue) = 2020
  0.081574 seconds (161.73 k allocations: 8.086 MiB, 24.40% gc time)
┌ Info: Answer found.
│   answer = 241861950
└ @ Main In[12]:4


In [14]:
@time show_answer_report(input_puzzle, Val(:part2), Val(:solution1))

entries_with_issue = three_entries_sum_to_2020(input, solution_val) = [1104, 715, 201]
sum(entries_with_issue) = 2020
  0.001274 seconds (149 allocations: 5.000 KiB)
┌ Info: Answer found.
│   answer = 158661360
└ @ Main In[12]:4
